# Running on Preimplantant embryos EMATB3929

Here we tested the dataset available at accession number: [E-MATB-3929](https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-3929/). Link to Paper: [Single-cell RNA-seq reveal lineage formation and X-chromosome dosage compensation in human preimplantation embryos](https://doi.org/10.1016/j.stemcr.2013.10.009)

<div id="toc"></div>

## Neccessary Imports

In [6]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [7]:
import sys
code = "./../../code/"
data = "./../../data/"
sys.path.append(code)
import pandas
import pypairs as pairs
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
import colorlover as cl
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import helper
import timeit

init_notebook_mode(connected=True)

In [8]:
cc_marker = helper.load_ocope_marker(data, fraction=0.6)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 10 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


## Counts

In [9]:
gencounts_EMTAB3929_counts = pandas.read_csv(Path(data + "E-MTAB-3929.processed.1_counts.txt"), sep='\t')

gencounts_EMTAB3929_counts.set_index("Unnamed: 0", inplace=True)
gencounts_EMTAB3929_counts.head(10)

,E5.5.101,E5.5.100,E6.2.114,E6.2.104,E6.2.107,E6.2.116,E7.2.138,E6.2.118,E6.2.105,E7.2.144,...,E3.50.3415,E3.51.3421,E3.53.3437,E3.51.3423,E3.52.3429,E3.49.3407,E3.51.3426,E3.47.3391,E3.52.3431,E3.53.3438
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,16,0,0,0,0,...,327,2167,170,451,104,446,2517,473,104,116
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,88,0,0,0,0,1,7,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,39,86,0,0,15,11,94,40,66,34
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,3,0,0,0,0,0,1,0,0,0,...,26,158,0,0,137,3,0,0,124,10
A2MP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
A4GALT,50,123,20,18,54,291,111,349,232,343,...,44,120,33,12,68,0,96,0,12,13


## Predicting

In [10]:
x = gencounts_EMTAB3929_counts.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_EMTAB3929_counts_Qnorm = pandas.DataFrame(X_std.T, index=gencounts_EMTAB3929_counts.index, columns=gencounts_EMTAB3929_counts.columns)

EMTAB3929_counts_prediction = pairs.cyclone(gencounts_EMTAB3929_counts, cc_marker, verbose=True, processes=0)

EMTAB3929_counts_prediction_table = helper.get_prediction_table(EMTAB3929_counts_prediction)
helper.DataTable(EMTAB3929_counts_prediction_table)

[__set_matrix] Original Matrix 'x' has shape 26178 x 1529
[__set_matrix] Matrix truncation done. Working with 26178 genes for 1529 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 229 marker pairs. 7917 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G2M: 948, G1: 196, S: 385


,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
E5.5.101,0.172,1.000,0.000,0.146758,0.853242,0.000000,G2M
E5.5.100,0.024,0.993,0.000,0.023599,0.976401,0.000000,G2M
E6.2.114,0.012,0.973,0.237,0.009820,0.796236,0.193944,G2M
E6.2.104,0.686,0.006,0.081,0.887451,0.007762,0.104787,G1
E6.2.107,0.062,0.906,0.105,0.057782,0.844362,0.097856,G2M
E6.2.116,0.187,0.799,0.006,0.188508,0.805444,0.006048,G2M
E7.2.138,0.479,1.000,0.000,0.323867,0.676133,0.000000,G2M
E6.2.118,0.081,0.553,0.686,0.061364,0.418939,0.519697,G2M
E6.2.105,0.175,0.684,0.051,0.192308,0.751648,0.056044,G2M


## Plotting prediction

In [11]:
classes = ["E3","E4.late","E4","E5.early","E5","E6","E7"]
labels = []
for idx in EMTAB3929_counts_prediction_table.index:
    for c in classes:
        if c in idx:
           labels.append(c) 
           break

In [12]:
e3_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],0].values
e3_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],1].values
print("E3 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],1].values)))

e4_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],0].values
e4_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],1].values
print("E4 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],1].values)))


e4late_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],0].values
e4late_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],1].values
print("E4 late {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],1].values)))


e5early_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],0].values
e5early_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],1].values
print("E5 early {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],1].values)))

e5_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],0].values
e5_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],1].values
print("E5 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],1].values)))

e6_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],0].values
e6_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],1].values
print("E6 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],1].values)))

e7_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],0].values
e7_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],1].values
print("E7 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],1].values)))

E3 81
E4 142
E4 late 48
E5 early 24
E5 353
E6 415
E7 466


In [13]:
set1 = cl.scales['9']['qual']['Set1']

e3_trace = go.Scatter(
    x = e3_g1,
    y = e3_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[1],
    ),
    name='E3'
)

e4_trace = go.Scatter(
    x = e4_g1,
    y = e4_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[2],
    ),
    name='E4'
)

e4late_trace = go.Scatter(
    x = e4late_g1,
    y = e4late_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[3],
    ),
    name='E4 late'
)


e5early_trace = go.Scatter(
    x = e5early_g1,
    y = e5early_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[4],
    ),
    name='E5 early'
)

e5_trace = go.Scatter(
    x = e5_g1,
    y = e5_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[5],
    ),
    name='E5'
)

e6_trace = go.Scatter(
    x = e6_g1,
    y = e6_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[6],
    ),
    name='E6'
)


e7_trace = go.Scatter(
    x = e7_g1,
    y = e7_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[7],
    ),
    name='E7'
)


lbls = go.Scatter(
    x=[0.8, 0.4, 0.25],
    y=[0.4, 0.8, 0.25],
    text=['G1',
          'G2M',
          'S'],
    mode='text',
    showlegend=False,
    hoverinfo='none'
)

data = [e3_trace, e4_trace, e4late_trace, e5early_trace, lbls]
#e5_trace, e6_trace, e7_trace,

layout = {
    'title': "Predicted cell cycle clustering for E3 - early E5",
    'xaxis': {
        'title': "G1 Score",
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'title': "G2M Score",
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': np.percentile(list(e3_g1) + list(e4_g1) + list(e4late_g1) + list(e5early_g1), 5),
            'y0': np.percentile(list(e3_g2m) + list(e4_g2m) + list(e4late_g2m) + list(e5early_g2m), 5),
            'x1': np.percentile(list(e3_g1) + list(e4_g1) + list(e4late_g1) + list(e5early_g1), 95),
            'y1': np.percentile(list(e3_g2m) + list(e4_g2m) + list(e4late_g2m) + list(e5early_g2m), 95),
            'opacity': 0.9,
            #'fillcolor': 'black',
            'line': {
                'color': 'red',
                'width': 5,
                'dash': 'dot'
            },
        },
        #{
        #    'type': 'circle',
        #    'xref': 'x',
        #    'yref': 'y',
        #    'x0': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 5),
        #    'y0': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 5),
        #    'x1': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 95),
        #    'y1': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 95),
        #    'opacity': 0.8,
        #    'fillcolor': 'blue',
        #    'line': {
        #        'color': 'black',
        #    },
        #},
    ]
}
fig = {
    'data': data,
    'layout': layout,
}

iplot(fig)

In [14]:
data = [e5_trace, e6_trace, e7_trace, lbls]
#

layout = {
    'title': "Predicted cell cycle clustering for E5 - E7",
    'xaxis': {
        'title': "G1 Score",
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'title': "G2M Score",
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 5),
            'y0': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 5),
            'x1': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 95),
            'y1': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 95),
            'opacity': 0.9,
            #'fillcolor': 'black',
            'line': {
                'color': 'red',
                'width': 5,
                'dash': 'dot'
            },
        },
    ]
}
fig = {
    'data': data,
    'layout': layout,
}

iplot(fig)

## Analyzing a single stage in depth

In [15]:
from collections import defaultdict

e3 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],:]
sub = defaultdict(list)
for index, row in e3.iterrows():
    pos = len(index) - 1 - index[::-1].index(".")
    s = index[:pos]
    sub[s].append((row[0], row[1], row[6]))
    
sub

defaultdict(list,
            {'E6.1': [(0.154, 0.157, 'S'),
              (0.443, 0.185, 'S'),
              (0.153, 0.077, 'S'),
              (0.239, 0.958, 'G2M'),
              (0.013, 0.115, 'S'),
              (0.885, 0.065, 'G1'),
              (0.107, 0.087, 'S'),
              (0.078, 0.579, 'G2M'),
              (0.119, 0.833, 'G2M'),
              (0.021, 0.029, 'S'),
              (0.094, 1.0, 'G2M'),
              (0.072, 1.0, 'G2M'),
              (0.719, 0.003, 'G1'),
              (0.8, 0.02, 'G1'),
              (0.563, 0.009, 'G1'),
              (0.05, 0.231, 'S')],
             'E6.10': [(0.895, 0.09, 'G1'),
              (0.969, 0.16, 'G1'),
              (0.252, 0.022, 'S'),
              (0.319, 0.213, 'S'),
              (0.351, 0.174, 'S'),
              (0.657, 0.009, 'G1'),
              (0.508, 0.216, 'G1'),
              (0.974, 0.002, 'G1'),
              (0.027, 0.54, 'G2M'),
              (0.534, 0.0, 'G1'),
              (0.215, 0.0, 'S'),
            

In [16]:
import colorlover as cl


bupu = cl.scales['7']['qual']['Set1']
bupu500 = cl.interp( bupu, 20 )

traces = []
i = 0
for key, val in sub.items():
    trace = go.Scatter(
        x = [i[0] for i in val],
        y = [i[1] for i in val],
        mode='markers+text',
        marker=dict(
            symbol='circle',
            size=10,
            color=bupu500[i],
        ),
        name=key
    )
    i+=1
    traces.append(trace)

layout = {
    'xaxis': {
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        }
    ]
}
fig = {
    'data': traces,
    'layout': layout,
}

iplot(fig)

## Stacked Bar Plot

In [17]:
e3_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],6].values)

e4_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],6].values)

e4late_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],6].values)

e5early_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],6].values)

e5_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],6].values)

e6_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],6].values)

e7_pred = list(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],6].values)

In [18]:
g1_trace = go.Bar(
    x=classes,
    y= [((e3_pred.count("G1") * 100) / len(e3_pred)),  
        ((e4_pred.count("G1") * 100) / len(e4_pred)),
        ((e4late_pred.count("G1") * 100) / len(e4late_pred)),
        ((e5early_pred.count("G1") * 100) / len(e5early_pred)),
        ((e5_pred.count("G1") * 100) / len(e5_pred)),
        ((e6_pred.count("G1") * 100) / len(e6_pred)),
        ((e7_pred.count("G1") * 100) / len(e7_pred))
       ],
    name='G1',
    marker=dict(
        color='blue',
    ),
)
s_trace = go.Bar(
    x=classes,
    y=[
        ((e3_pred.count("S") * 100) / len(e3_pred)),  
        ((e4_pred.count("S") * 100) / len(e4_pred)),
        ((e4late_pred.count("S") * 100) / len(e4late_pred)),
        ((e5early_pred.count("S") * 100) / len(e5early_pred)),
        ((e5_pred.count("S") * 100) / len(e5_pred)),
        ((e6_pred.count("S") * 100) / len(e6_pred)),
        ((e7_pred.count("S") * 100) / len(e7_pred))
    ],
    name='S',
    marker=dict(
        color='orange',
    ),
)
g2m_trace = go.Bar(
    x=classes,
    y=[
        ((e3_pred.count("G2M") * 100) / len(e3_pred)),  
        ((e4_pred.count("G2M") * 100) / len(e4_pred)),
        ((e4late_pred.count("G2M") * 100) / len(e4late_pred)),
        ((e5early_pred.count("G2M") * 100) / len(e5early_pred)),
        ((e5_pred.count("G2M") * 100) / len(e5_pred)),
        ((e6_pred.count("G2M") * 100) / len(e6_pred)),
        ((e7_pred.count("G2M") * 100) / len(e7_pred))
    ],
    name='G2M',
    marker=dict(
        color='green',
    ),
)


data = [g1_trace, s_trace, g2m_trace]
layout = go.Layout(
    barmode='stack',
    xaxis=dict(title="Ebryonic Stage"),
    yaxis=dict(title="% of cells"),
    title="Distribution of predicted phases per embryonic stage"
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

## Per lineage

In [21]:
lineage_map = {}

header = True
for line in open("./../../data/E-MTAB-3929_annotation.txt"):
    if header:
        header = False
        continue
    info = line.split("\t")
    if info[12] != 'not applicable':
        lineage_map[info[0]] = info[12]
    
lineage_map

{'E5.1.26': 'epiblast',
 'E5.1.28': 'trophectoderm',
 'E5.1.30': 'trophectoderm',
 'E5.1.31': 'primitive endoderm',
 'E5.1.32': 'trophectoderm',
 'E5.10.933': 'trophectoderm',
 'E5.10.934': 'trophectoderm',
 'E5.10.935': 'trophectoderm',
 'E5.10.936': 'trophectoderm',
 'E5.10.937': 'trophectoderm',
 'E5.10.938': 'primitive endoderm',
 'E5.10.939': 'trophectoderm',
 'E5.10.940': 'epiblast',
 'E5.10.942': 'primitive endoderm',
 'E5.10.943': 'trophectoderm',
 'E5.10.944': 'trophectoderm',
 'E5.10.945': 'trophectoderm',
 'E5.10.947': 'epiblast',
 'E5.10.948': 'epiblast',
 'E5.10.949': 'trophectoderm',
 'E5.10.950': 'epiblast',
 'E5.11.951': 'trophectoderm',
 'E5.11.952': 'trophectoderm',
 'E5.11.956': 'epiblast',
 'E5.11.957': 'epiblast',
 'E5.11.959': 'epiblast',
 'E5.11.960': 'epiblast',
 'E5.11.962': 'epiblast',
 'E5.11.963': 'epiblast',
 'E5.11.964': 'epiblast',
 'E5.12.1027': 'trophectoderm',
 'E5.12.1028': 'trophectoderm',
 'E5.12.1029': 'trophectoderm',
 'E5.12.1031': 'trophectoderm

In [27]:
e5_epiblast = [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'epiblast' and "E5." in c] 
e6_epiblast =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'epiblast' and "E6." in c]
e7_epiblast =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'epiblast' and "E7." in c]

e5_primitive = [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'primitive endoderm' and "E5." in c] 
e6_primitive =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'primitive endoderm' and "E6." in c]
e7_primitive =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'primitive endoderm' and "E7." in c] 

e5_trophectoderm = [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'trophectoderm' and "E5." in c] 
e6_trophectoderm =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'trophectoderm' and "E6." in c]
e7_trophectoderm =  [EMTAB3929_counts_prediction_table.loc[c, :].values[6] for c, t in lineage_map.items() if t == 'trophectoderm' and "E7." in c] 

In [49]:
g1_e5_trace = go.Bar(
    x = ["E5 EB", "E5 PE", "E5 TR"],
    y = [
        ((e5_epiblast.count("G1") * 100) / len(e5_epiblast)),
        ((e5_primitive.count("G1") * 100) / len(e5_primitive)),
        ((e5_trophectoderm.count("G1") * 100) / len(e5_trophectoderm)),
    ],
    name='G1',
    marker=dict(
        color='blue',
    )
)



s_e5_trace = go.Bar(
    x = ["E5 EB", "E5 PE", "E5 TR"],
    y = [
        ((e5_epiblast.count("S") * 100) / len(e5_epiblast)),
        ((e5_primitive.count("S") * 100) / len(e5_primitive)),
        ((e5_trophectoderm.count("S") * 100) / len(e5_trophectoderm)),
    ],
    name='S',
    marker=dict(
        color='orange',
    )
)

g2m_e5_trace = go.Bar(
    x = ["E5 EB", "E5 PE", "E5 TR"],
    y = [
        ((e5_epiblast.count("G2M") * 100) / len(e5_epiblast)),
        ((e5_primitive.count("G2M") * 100) / len(e5_primitive)),
        ((e5_trophectoderm.count("G2M") * 100) / len(e5_trophectoderm)),
    ],
    name='G2M',
    marker=dict(
        color='green',
    )
)

g1_e6_trace = go.Bar(
    x = ["E6 EB", "E6 PE", "E6 TR"],
    y = [
        ((e6_epiblast.count("G1") * 100) / len(e6_epiblast)),
        ((e6_primitive.count("G1") * 100) / len(e6_primitive)),
        ((e6_trophectoderm.count("G1") * 100) / len(e6_trophectoderm)),
    ],
    name='G1',
    marker=dict(
        color='blue',
    )
)



s_e6_trace = go.Bar(
    x = ["E6 EB", "E6 PE", "E6 TR"],
    y = [
        ((e6_epiblast.count("S") * 100) / len(e6_epiblast)),
        ((e6_primitive.count("S") * 100) / len(e6_primitive)),
        ((e6_trophectoderm.count("S") * 100) / len(e6_trophectoderm)),
    ],
    name='S',
    marker=dict(
        color='orange',
    )
)

g2m_e6_trace = go.Bar(
    x = ["E6 EB", "E6 PE", "E6 TR"],
    y = [
        ((e6_epiblast.count("G2M") * 100) / len(e6_epiblast)),
        ((e6_primitive.count("G2M") * 100) / len(e6_primitive)),
        ((e6_trophectoderm.count("G2M") * 100) / len(e6_trophectoderm)),
    ],
    name='G2M',
    marker=dict(
        color='green',
    )
)

g1_e7_trace = go.Bar(
    x = ["E7 EB", "E7 PE", "E7 TR"],
    y = [
        ((e7_epiblast.count("G1") * 100) / len(e7_epiblast)),
        ((e7_primitive.count("G1") * 100) / len(e7_primitive)),
        ((e7_trophectoderm.count("G1") * 100) / len(e7_trophectoderm)),
    ],
    name='G1',
    marker=dict(
        color='blue',
    )
)



s_e7_trace = go.Bar(
    x = ["E7 EB", "E7 PE", "E7 TR"],
    y = [
        ((e7_epiblast.count("S") * 100) / len(e7_epiblast)),
        ((e7_primitive.count("S") * 100) / len(e7_primitive)),
        ((e7_trophectoderm.count("S") * 100) / len(e7_trophectoderm)),
    ],
    name='S',
    marker=dict(
        color='orange',
    )
)

g2m_e7_trace = go.Bar(
    x = ["E7 EB", "E7 PE", "E7 TR"],
    y = [
        ((e7_epiblast.count("G2M") * 100) / len(e7_epiblast)),
        ((e7_primitive.count("G2M") * 100) / len(e7_primitive)),
        ((e7_trophectoderm.count("G2M") * 100) / len(e7_trophectoderm)),
    ],
    name='G2M',
    marker=dict(
        color='green',
    )
)

data = [g1_e5_trace, s_e5_trace, g2m_e5_trace, g1_e6_trace, s_e6_trace, g2m_e6_trace, g1_e7_trace, s_e7_trace, g2m_e7_trace]

layout = {
    'barmode':'stack',
    'xaxis': {
        'title':"Embryonic Stage",
    },
    'yaxis': {
        'title':"% of cells"
    },
    'title':"Distribution of predicted phases per embryonic stage",
    'shapes': [
        {
            'type': 'path',
            'path': ' M 2.5,0 L2.5,110',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        {
            'type': 'path',
            'path': ' M 5.5,0 L5.5,110',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        }
    ]
}
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar', image='svg')